In [2]:
import networkx as nx
import numpy as np

In [34]:
def propagate_neighbours(graph:nx.Graph, new_active, p, seed=None):
    neighbors = []
    for node in new_active:
        neighbors += g.neighbors(node)
    
    if seed:
        np.random.seed(seed)

    activation = np.random.uniform(low=0, high=1, size=len(neighbors)) < p
    activated_neighbors = list(np.extract(activation, sorted(neighbors)))

    return activated_neighbors

def independent_cascade_model(graph:nx.Graph, activated_nodes, p=.2, seed=None):
    new_active = activated_nodes[:]
    list_active = activated_nodes[:]
    while new_active:
        activated_neighbors = propagate_neighbours(graph, new_active, p=p, seed=seed)
        new_active = list(set(activated_neighbors) - set(list_active))
        list_active  += new_active
    
    spread = len(list_active)

    return {'spread':spread, 'list_active':list_active}

def greedy_influence_maximisation(g:nx.Graph, k, p=.2, seed=None):
    A0 = []
    for node_idx in range(k):
        nodes = set(g.nodes()) - set(A0)
        spread_score = {node:independent_cascade_model(g, A0 + [node], p=p, seed=seed) for node in g.nodes()}
        max_node = -1; max_spread = -1
        for n, d in spread_score.items():
            if d['spread'] > max_spread:
                max_node = n
                max_spread = d['spread']
        A0.append(max_node)
    list_active = spread_score.get(A0[-1])['list_active']
    return {'A0':A0, 'spread':len(list_active), 'list_active':list_active}


In [45]:
g = nx.karate_club_graph()

info_icm = independent_cascade_model(g, [0], p=.1)
print(info_icm)

info= greedy_influence_maximisation(g, 3)
print(A0)

{'spread': 1, 'list_active': [0]}
{'A0': [20, 1, 20], 'spread': 23, 'list_active': [20, 1, 20, 3, 13, 30, 0, 33, 12, 32, 5, 19, 21, 23, 26, 29, 31, 8, 25, 10, 22, 2, 27]}
